In [18]:
# 03_fundamental_filters.ipynb
# Author: André Lopes Marinho
# Description:
# This notebook collects fundamental indicators for Brazilian stocks from Fundamentus,
# applies quality filters, and saves a list of healthy companies for portfolio analysis.

# Step 1: Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import unicodedata
import io

# Step 2: Download and parse the table from Fundamentus
url = 'https://www.fundamentus.com.br/resultado.php'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table')

# Step 3: Read HTML table into a DataFrame
df = pd.read_html(io.StringIO(str(table)), decimal=',', thousands='.')[0]

# Step 4: Clean column names (remove accents, symbols, spaces, make lowercase)
def clean_column(col):
    col = unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8')
    col = col.lower().strip()
    col = col.replace(' ', '_').replace('.', '').replace('/', '_').replace('%', 'pct')
    return col

df.columns = [clean_column(col) for col in df.columns]

# Step 5: Convert selected columns to numeric (trata também % escondidos)
def convert_column(col):
    # Remove "%" se existir, troca vírgula por ponto (extra segurança)
    df[col] = df[col].astype(str).str.replace('%', '', regex=False).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in ['p_l', 'roe', 'divyield', 'divbrut__patrim', 'liq_corr']:
    convert_column(col)

# Step 6: Apply quality filters
# We'll use common filters such as:
# - P/L (price to earnings ratio) between 0 and 20
# - ROE (return on equity) > 10%
# - Low debt (div_liq_patrim < 1)
filtered = df[
    (df['roe'] > 10) &
    (df['divyield'] > 3) &
    (df['p_l'] > 0) & (df['p_l'] < 20) &
    (df['divbrut__patrim'] < 1.0) &
    (df['liq_corr'] > 1.0)
]

# Step 7: Sort and display results
filtered = filtered.sort_values('roe', ascending=False)

print(df[['papel', 'roe', 'divyield', 'p_l', 'divbrut__patrim', 'liq_corr']].describe())

if filtered.empty:
    print("No companies matched the criteria.")
else:
    print("Top fundamental stocks based on filters:")
    display(filtered[['papel', 'roe', 'p_l', 'divyield', 'divbrut__patrim']].head(10))


print("Top fundamental stocks based on filters:")
display(filtered[['papel', 'roe', 'p_l', 'divyield', 'divbrut__patrim']].head(10))

# Step 8: Save filtered results to CSV
filtered.to_csv('../data/fundamental_filtered_stocks.csv', index=False)


              roe    divyield          p_l  divbrut__patrim    liq_corr
count  984.000000  991.000000   992.000000       992.000000  992.000000
mean    10.209207    2.358426    -4.388931        44.319738    1.897218
std     87.529791    6.939545   331.912681       952.751228    4.409499
min   -889.480000    0.000000 -7342.320000       -49.850000    0.000000
25%      0.000000    0.000000    -0.257500         0.000000    0.475000
50%      9.505000    0.000000     4.685000         0.280000    1.300000
75%     19.840000    2.390000    11.802500         1.110000    2.100000
max    725.280000  121.490000  1719.330000     21227.000000   66.470000
Top fundamental stocks based on filters:


,papel,roe,p_l,divyield,divbrut__patrim
412,TRPN3,122.35,0.44,104.74,0.00
432,SYNE3,51.13,1.58,121.49,0.77
623,PLPL3,46.07,8.12,6.84,0.75
639,CMIN3,35.30,8.53,14.77,0.96
492,SOND6,33.27,4.44,14.58,0.00
504,SOND3,33.27,4.85,12.13,0.00
469,SOND5,33.27,3.79,17.08,0.00
624,WHRL4,32.67,8.16,13.48,0.41
590,WHRL3,32.67,7.42,13.48,0.41
700,DIRR3,29.96,10.22,8.66,0.91


Top fundamental stocks based on filters:


,papel,roe,p_l,divyield,divbrut__patrim
412,TRPN3,122.35,0.44,104.74,0.00
432,SYNE3,51.13,1.58,121.49,0.77
623,PLPL3,46.07,8.12,6.84,0.75
639,CMIN3,35.30,8.53,14.77,0.96
492,SOND6,33.27,4.44,14.58,0.00
504,SOND3,33.27,4.85,12.13,0.00
469,SOND5,33.27,3.79,17.08,0.00
624,WHRL4,32.67,8.16,13.48,0.41
590,WHRL3,32.67,7.42,13.48,0.41
700,DIRR3,29.96,10.22,8.66,0.91
